In [1]:
import pandas as pd
import requests
from pandas_profiling import ProfileReport
import sweetviz as sv
import numpy as np

In [2]:
unnecesary_columns = ['V_ID','P_ID','C_SEV','C_YEAR','P_USER','P_PSN','V_YEAR','P_ISEV']

In [3]:
data = pd.read_csv('NCDB_1999_to_2014.csv',low_memory=False)#.drop(unnecesary_columns,1)

In [4]:
data.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,01,1,20,2,02,34,UU,1,5,...,01,06,1990,01,M,41,11,1,UU,1
1,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,01,M,19,11,1,UU,1
2,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,02,F,20,13,2,02,2
3,1999,01,1,08,2,01,01,UU,5,3,...,01,01,1986,01,M,46,11,1,UU,1
4,1999,01,1,08,2,01,01,UU,5,3,...,99,NN,NNNN,01,M,05,99,2,UU,3


In [5]:
c_groups = [col for col in data.columns if col[0]=="C"]
#c_groups.append('P_ID')
df_columns = [col for col in data.columns if col[0]=="C"]
df_columns.append('P_ID')
data = data.merge(data[df_columns].groupby(df_columns).size().reset_index().drop('P_ID',axis=1).rename(columns={0:"A_PERS"}),on=c_groups)

In [6]:
print(f'Numero de registros: {data.shape[0]}')
print(f'Numero de columnas: {data.shape[1]}')

Numero de registros: 13672837
Numero de columnas: 23


In [7]:
data.dtypes.to_dict()

{'C_YEAR': dtype('int64'),
 'C_MNTH': dtype('O'),
 'C_WDAY': dtype('O'),
 'C_HOUR': dtype('O'),
 'C_SEV': dtype('int64'),
 'C_VEHS': dtype('O'),
 'C_CONF': dtype('O'),
 'C_RCFG': dtype('O'),
 'C_WTHR': dtype('O'),
 'C_RSUR': dtype('O'),
 'C_RALN': dtype('O'),
 'C_TRAF': dtype('O'),
 'V_ID': dtype('O'),
 'V_TYPE': dtype('O'),
 'V_YEAR': dtype('O'),
 'P_ID': dtype('O'),
 'P_SEX': dtype('O'),
 'P_AGE': dtype('O'),
 'P_PSN': dtype('O'),
 'P_ISEV': dtype('O'),
 'P_SAFE': dtype('O'),
 'P_USER': dtype('O'),
 'A_PERS': dtype('int64')}

In [8]:
nas = ["UU","XX","U","X","XXXX","NN","NNNN","N","UUUU"]
others = {
    "P_SAFE" : "14",
    "P_PSN": "100",
    "V_TYPE": "24",
    "C_TRAF": "19",
    "C_RALN": "7",
    "C_RSUR": "10",
    "C_WTHR": "8",
    "C_RCFG": "13",
    "C_CONF": "42"
}
int_types = ["A_DAGE","A_VAGE","A_PERS"]
categories = [col for col in data.columns if col not in int_types]
target_columns = ["C_SEV","P_ISEV"]
#ver q hacer con este C_HOUR

In [9]:
for na in nas:
    data[categories] = data[categories].replace(na,pd.NA)

for key,value in others.items():
    data[key] = data[key].replace(to_replace=r'^Q.*', value=value, regex=True)
    
#data.replace(to_replace=r'^[A-Z].*',value=np.nan, regex=True,inplace=True)

F:\Anaconda\envs\python37v2\lib\site-packages\pandas\core\missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


In [10]:
#data[int_types] = data[int_types].apply(pd.to_numeric, errors='coerce',downcast='integer')
#data[data["V_YEAR"]=="NNNN"]["V_YEAR"]

In [11]:
#data['V_YEAR'].replace("NNNN",pd.NA,inplace=True)
data['A_VAGE'] = data['C_YEAR'] - pd.to_numeric(data['V_YEAR'], errors='coerce')

In [12]:
#quedarse solo con los drivers
data = data[data['P_PSN']=="11"]

In [13]:
def transform_hour(value):
    value = pd.to_numeric(value,errors='coerce')
    if value >= 0 and value<6:
        return '0'
    if value >= 6 and value<12:
        return '1'
    if value >= 12 and value<18:
        return '2'
    if value >= 18 and value<24:
        return '3'
    return pd.NA

data['C_HOUR'] = data['C_HOUR'].apply(transform_hour) 
    

In [14]:
#data[int_types].apply(pd.to_numeric, errors='coerce',downcast='integer')
#data[int_types] = data[int_types].astype(pd.Int64Dtype(),errors='ignore')
data[categories] = data[categories].astype('category')
data.dtypes.to_dict()

{'C_YEAR': CategoricalDtype(categories=[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
                   2009, 2010, 2011, 2012, 2013, 2014],
                  ordered=False),
 'C_MNTH': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '11', '12'],
                  ordered=False),
 'C_WDAY': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7', 'U'], ordered=False),
 'C_HOUR': CategoricalDtype(categories=['0', '1', '2', '3'], ordered=False),
 'C_SEV': CategoricalDtype(categories=[1, 2], ordered=False),
 'C_VEHS': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
                   '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
                   '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
                   '41', '43', '44', '46', '51', '54', '56', '57', '58'

In [15]:
data = data.rename(columns={'P_SEX':'A_DSEX','P_AGE':"A_DAGE",'C_HOUR':'A_CHUR'})

In [16]:
data[int_types] = data[int_types].apply(pd.to_numeric, errors='coerce',downcast='integer')

In [17]:
data = data.drop(unnecesary_columns,1)

In [18]:
data.dtypes.to_dict()

{'C_MNTH': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '11', '12'],
                  ordered=False),
 'C_WDAY': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7', 'U'], ordered=False),
 'A_CHUR': CategoricalDtype(categories=['0', '1', '2', '3'], ordered=False),
 'C_VEHS': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
                   '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
                   '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
                   '41', '43', '44', '46', '51', '54', '56', '57', '58', '71',
                   '72', '77'],
                  ordered=False),
 'C_CONF': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '21', '22', '23', '24',
                   '25', '31', '32', '33', '34', '35', '36', '41', '42'],
    

In [ ]:
#my_report = sv.analyze(data)
#my_report.show_notebook()
profile = ProfileReport(data, title="Crashes EDA")
profile.to_widgets()

In [ ]:
#data["C_MNTH"].unique()

In [ ]:
#data["P_ISEV"].isna().sum()

In [ ]:
#data["A_VAGE"].isna().sum()

In [ ]:
#data[data["A_PCOUNT"]==77]